# Notebook for handling disorder in CIFs

Distributed under the terms of the MIT License.

Author: Andrew Tarzia

Date Created: 05 Mar 2019


## This is a testing ground.

In [64]:
from pymatgen.io.cif import CifParser
from pymatgen.util.coord import in_coord_list_pbc, pbc_diff, \
    find_in_coord_list_pbc
import numpy as np
import pymatgen as pmg
from collections import OrderedDict

In [65]:
CIF_DIR = '/home/atarzia/projects/cage_collect/databases/cage_manual/'

In [66]:
CIF = 'KOJWUC_extracted.cif'

In [67]:
CF = CifParser(CIF_DIR+CIF, occupancy_tolerance=100)
st = CF.get_structures(primitive=False)[0]

/home/atarzia/anaconda3/lib/python3.6/site-packages/pymatgen/io/cif.py:1000: UserWarning: Some occupancies ([12, 12, 12, 12, 48, 48, 48, 24, 24, 48, 48, 48, 24, 24]) sum to > 1! If they are within the tolerance, they will be rescaled.
  warnings.warn(msg)
/home/atarzia/anaconda3/lib/python3.6/site-packages/pymatgen/io/cif.py:1108: UserWarning: Some occupancies ([12, 12, 12, 12, 48, 48, 48, 24, 24, 48, 48, 48, 24, 24]) sum to > 1! If they are within the tolerance, they will be rescaled.
  warnings.warn(error)


In [68]:
st

Structure Summary
Lattice
    abc : 29.663 29.663 29.663
 angles : 90.0 90.0 90.0
 volume : 26100.282937247
      A : 29.663 0.0 1.816334900155397e-15
      B : 4.770174215735807e-15 29.663 1.816334900155397e-15
      C : 0.0 0.0 29.663
PeriodicSite: Cu (14.8315, 7.3045, 7.3045) [0.5000, 0.2462, 0.2462]
PeriodicSite: Cu (7.3045, 14.8315, 7.3045) [0.2462, 0.5000, 0.2462]
PeriodicSite: Cu (7.3045, 7.3045, 14.8315) [0.2462, 0.2462, 0.5000]
PeriodicSite: Cu (7.3045, 14.8315, 22.3585) [0.2462, 0.5000, 0.7538]
PeriodicSite: Cu (22.3585, 14.8315, 7.3045) [0.7538, 0.5000, 0.2462]
PeriodicSite: Cu (14.8315, 7.3045, 22.3585) [0.5000, 0.2462, 0.7538]
PeriodicSite: Cu (14.8315, 22.3585, 7.3045) [0.5000, 0.7538, 0.2462]
PeriodicSite: Cu (7.3045, 22.3585, 14.8315) [0.2462, 0.7538, 0.5000]
PeriodicSite: Cu (22.3585, 7.3045, 14.8315) [0.7538, 0.2462, 0.5000]
PeriodicSite: Cu (0.0000, 22.1360, 22.1360) [0.0000, 0.7462, 0.7462]
PeriodicSite: Cu (22.1360, 0.0000, 22.1360) [0.7462, 0.0000, 0.7462]
Periodi

In [72]:
a = st[2]

In [78]:
a.c

0.5

### below I look at making sure atoms with matching coordinates are removed (this is handled in CIFPARSER)

In [46]:
def get_matching_coord(structure, coord, coord_to_species, cifparse, site_tolerance):
    '''Find atoms with matching coordinates within some tolerance.
    
    Modified from the function in pymatgen.io.cif.py by the same name.
    
    '''
    keys = list(coord_to_species.keys())
    coords = np.array(keys)
    for op in cifparse.symmetry_operations:
        c = op.operate(coord)
        inds = find_in_coord_list_pbc(coords, c,
                                      atol=site_tolerance)
        # cant use if inds, because python is dumb and np.array([0]) evaluates
        # to False
        if len(inds):
            return keys[inds[0]]
    return False

In [47]:
site = st[6]

In [48]:
site.species_and_occu

Comp: Ag1

In [49]:
coord_to_species = {}
for site in st:
    coord = (site.x, site.y, site.z)
    site_tolerance = 1e-4
    match = get_matching_coord(structure=st, coord=coord, coord_to_species=coord_to_species, 
                               cifparse=CF, site_tolerance=site_tolerance)
    if match is False:
        coord_to_species[tuple(site.coords)] = site.species_and_occu
    else:
        coord_to_species[match] += site.species_and_occu
        print('adding')
        print(site, match)

In [45]:
st.to(filename=CIF_DIR+'test.cif', fmt='cif') 

In [51]:
len(coord_to_species)

710

In [52]:
len(st)

710